In [0]:
from google.colab import drive

MOUNT_PATH = "/content/gdrive"
drive.mount(MOUNT_PATH)

DATA_PATH = MOUNT_PATH + "/My Drive/Kaggle/data"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:

# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x58eae000 @  0x7f70307702a4 0x591a07 0x5b5d56 0x502e9a 0x506859 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x502209 0x502f3d 0x506859 0x504c28 0x502540 0x502f3d 0x507641 0x504c28 0x502540 0x502f3d 0x507641


In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import csv
from google.colab import files

number_of_train = 7000
batch_Size = 150
number_of_epochs = 200

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.drop = nn.Dropout2d(p=0.05)
        self.dropfc = nn.Dropout(p=0.20)
        self.conv1 = nn.Conv2d(1, 128, 5, padding=0) 
        self.conv2 = nn.Conv2d(128, 256, 5, padding=0)
        self.conv3 = nn.Conv2d(256, 256, 5, padding=0)
        self.conv4 = nn.Conv2d(256, 128, 5, padding=0)
        self.bn1 = nn.BatchNorm2d(128)        
        self.bn2 = nn.BatchNorm2d(256)
        self.bn3 = nn.BatchNorm2d(256)
        self.bn4 = nn.BatchNorm2d(128)
        self.bn5 = nn.BatchNorm2d(92)
        self.bn6 = nn.BatchNorm1d(1024)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 5 * 5, 1024)
        self.fc2 = nn.Linear(1024, 31)

    def forward(self, x):
        x = self.drop(self.pool(self.bn1(F.relu(self.conv1(x)))))
        x = self.drop(self.pool(self.bn2(F.relu(self.conv2(x))))) 
        x = self.drop(self.pool(self.bn3(F.relu(self.conv3(x)))))
        x = self.drop(self.bn4(F.relu(self.conv4(x))))
        x = x.view(-1, 128 * 5 * 5)
        x = self.dropfc(self.bn6(F.relu(self.fc1(x))))
        x = self.fc2(x)
        return x

class custom_dataset(torch.utils.data.dataset.Dataset):
    def __init__(self, data, labels, transforms=None):
        self.data = data
        self.labels = labels
        self.transforms = transforms

    def __getitem__(self, index):
        dat = self.data[index]
        if self.transforms is not None:
            dat = self.transforms(dat)
        return (dat,self.labels[index])
   
    def __len__(self):
        return len(self.data)

def define_classes(labels):
    classes = {}
    label = []
    c = 0
    for i,j in labels:
        if j.decode('utf-8') not in classes:
            classes[j.decode('utf-8')] = c
            c += 1
        label.append(classes[j.decode('utf-8')])    
    return torch.LongTensor(label)

################################################################################
X_train_final = []
X_test_final = []
################################################################################
def train(images, labels):
    
    transform = transforms.Compose(
        [transforms.ToPILImage(), transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset = custom_dataset(X_train_final[:number_of_train],define_classes(labels)[:number_of_train], transforms=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_Size, shuffle=True, num_workers=0)
    
    validation_set = custom_dataset(X_train_final[number_of_train:],define_classes(labels)[number_of_train:],transforms=transform)
    validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_Size, shuffle=False, num_workers=0)

    model = ConvNet().to(device)
    criterion = nn.CrossEntropyLoss(weight=weight(define_classes(labels)[:number_of_train]).to(device))
    optimizer = optim.SGD(model.parameters(), lr=0.025, weight_decay=0.015, momentum=0.9) 
    

    train_accuracy = []
    test_accuracy = []
    print("Training accuracy for: epochs #", number_of_epochs)
    for epoch in range(number_of_epochs):
        if epoch>20:
            optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0.015, momentum=0.9) 
        if epoch>30: 
            optimizer = optim.SGD(model.parameters(), lr=0.001, weight_decay=0.015, momentum=0.9) 
        if epoch>40:
            optimizer = optim.SGD(model.parameters(), lr=0.0005, weight_decay=0.015, momentum=0.9)
            
        correct = 0.
        total = 0.
        model.train()
        for i, data in enumerate(trainloader, 0):
            
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs.to(device))
            loss = criterion(outputs, labels.to(device))
            loss.backward()
            optimizer.step()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels.to(device)).sum().item()
        train_accuracy.append(correct / total)
        
        correct = 0.
        total = 0.
        model.eval()
        with torch.no_grad():
            for data in validation_loader:
                images, labels = data
                outputs = model(images.to(device))
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels.to(device)).sum().item()
                loss = criterion(outputs, labels.to(device))
        test_accuracy.append(correct/total)
        
        print('Epoch:', epoch)
        print('Train accuracy:', train_accuracy[epoch]*100)
        print('Test accuracy:', test_accuracy[epoch]*100)        
    return model

def weight(labels):
    scale = torch.FloatTensor(31)
    for i in range(31):
        scale[i] = ((labels==i).sum())
    return scale.max() / scale

if __name__ == '__main__':
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    torch.cuda.manual_seed(10)
    images_train = np.load(DATA_PATH+'/train_images.npy', encoding='latin1')
    images_test = np.load(DATA_PATH+'/test_images.npy', encoding='latin1')

    train_labels = np.genfromtxt(DATA_PATH+'/train_labels.csv', names=True, delimiter=',', dtype=[('Id', 'i8'), ('Category', 'S5')])
    for i in range(0, len(images_train)):
        X_train_final.append(torch.Tensor(images_train[i, 1].reshape((1, 100, 100)))/255)
    for i in range(0, len(images_test)):
        X_test_final.append(torch.Tensor(images_test[i, 1].reshape((1, 100, 100)))/255)
    model = train(images_train, train_labels)
    transform_test = transforms.Compose([transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    testdata_set = custom_dataset(X_test_final,torch.tensor([0.]*len(X_test_final)),transforms=transform_test)
    
    #########################################################################################################################
    #Validation
    #transform = transforms.Compose([transforms.ToPILImage(), transforms.RandomHorizontalFlip(),transforms.RandomVerticalFlip(),transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    #validation_set = custom_dataset(X_train_final[number_of_train:],define_classes(train_labels)[number_of_train:],transforms=transform)
    #validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_Size, shuffle=False, num_workers=0)
    
   
    #correct = 0
    #total = 0
    #model.eval()
    #with torch.no_grad():
        #for i in range(10):
            #for data in validation_loader:
                #images, labels = data
                #outputs = model(images.to(device))
                #_, predicted = torch.max(outputs.data, 1)
                #total += labels.size(0)
                #correct += (predicted == labels.to(device)).sum().item()
        #print("Test accuracy 2000 samples:", (correct/total)*100)
    #########################################################################################################################
    
    #####################################################################################################################
    #Prediction
    test_loader = torch.utils.data.DataLoader(testdata_set, batch_size=batch_Size, shuffle=False, num_workers=0)
    image_dict = {0:'shovel', 1:'rifle' ,2:'scorpion', 3:'apple', 4:'spoon', 5:'pineapple', 6:'mouth', 7:'skateboard',
    8:'rollerskates',9:'peanut', 10:'rabbit', 11:'sink',12:'sailboat', 13:'nose', 14:'skull',15:'pool',16:'pear',17:'pillow',
    18:'penguin', 19:'nail', 20:'pencil', 21:'empty', 22:'octagon', 23:'moustache', 24:'paintbrush', 25:'panda', 26:'parrot', 27:'screwdriver',
    28:'squiggle', 29:'rhinoceros', 30:'mug'}

    all_prections = []
    model.eval()
    with torch.no_grad():
        for i, (images, labels) in enumerate(test_loader):
            test_pred = model(images.to(device))
            _, predicted = torch.max(test_pred.data, 1)
            all_prections.extend(predicted)

    with open(r'final_submission.csv', mode='w', newline='') as submission:
            writer = csv.writer(submission, delimiter=',',
                quotechar='"', quoting=csv.QUOTE_MINIMAL) 
            writer.writerow(['Id', 'Category'])
            for i in range(len(all_prections)):
                all_prections[i] = image_dict[all_prections[i].item()]
                writer.writerow([str(i), all_prections[i]])
    ########################################################################################################################
#     files.download(r'final_submission.csv')
    torch.cuda.empty_cache()


Training accuracy for: epochs # 200
Epoch: 0
Train accuracy: 4.857142857142857
Test accuracy: 3.5666666666666664
Epoch: 1
Train accuracy: 7.142857142857142
Test accuracy: 4.466666666666667
Epoch: 2
Train accuracy: 11.085714285714285
Test accuracy: 9.700000000000001
Epoch: 3
Train accuracy: 20.3
Test accuracy: 11.533333333333333
Epoch: 4
Train accuracy: 26.971428571428575
Test accuracy: 22.866666666666667
Epoch: 5
Train accuracy: 29.34285714285714
Test accuracy: 29.333333333333332
Epoch: 6
Train accuracy: 33.41428571428571
Test accuracy: 22.26666666666667
Epoch: 7
Train accuracy: 35.92857142857142
Test accuracy: 15.266666666666667
Epoch: 8
Train accuracy: 38.31428571428571
Test accuracy: 30.3
Epoch: 9
Train accuracy: 40.85714285714286
Test accuracy: 37.766666666666666
Epoch: 10
Train accuracy: 42.871428571428574
Test accuracy: 21.666666666666668
Epoch: 11
Train accuracy: 43.628571428571426
Test accuracy: 32.93333333333333
Epoch: 12
Train accuracy: 45.7
Test accuracy: 36.6
Epoch: 13
Trai

In [0]:
train = [4.857142857142857,7.142857142857142, 11.085714285714285, 20.3, 26.971428571428575, 29.34285714285714, 33.41428571428571, 35.92857142857142, 
Test accuracy: 15.266666666666667
Epoch: 8
Train accuracy: 38.31428571428571
Test accuracy: 30.3
Epoch: 9
Train accuracy: 40.85714285714286
Test accuracy: 37.766666666666666
Epoch: 10
Train accuracy: 42.871428571428574
Test accuracy: 21.666666666666668
Epoch: 11
Train accuracy: 43.628571428571426
Test accuracy: 32.93333333333333
Epoch: 12
Train accuracy: 45.7
Test accuracy: 36.6
Epoch: 13
Train accuracy: 45.68571428571428
Test accuracy: 21.96666666666667
Epoch: 14
Train accuracy: 47.08571428571429
Test accuracy: 26.833333333333332
Epoch: 15
Train accuracy: 48.714285714285715
Test accuracy: 42.6
Epoch: 16
Train accuracy: 49.857142857142854
Test accuracy: 39.900000000000006
Epoch: 17
Train accuracy: 51.91428571428571
Test accuracy: 44.5
Epoch: 18
Train accuracy: 52.485714285714295
Test accuracy: 27.53333333333333
Epoch: 19
Train accuracy: 53.900000000000006
Test accuracy: 39.300000000000004
Epoch: 20
Train accuracy: 53.87142857142857
Test accuracy: 33.33333333333333
Epoch: 21
Train accuracy: 58.91428571428572
Test accuracy: 58.36666666666667
Epoch: 22
Train accuracy: 62.68571428571429
Test accuracy: 59.833333333333336
Epoch: 23
Train accuracy: 63.857142857142854
Test accuracy: 60.53333333333333
Epoch: 24
Train accuracy: 65.4
Test accuracy: 62.133333333333326
Epoch: 25
Train accuracy: 66.37142857142857
Test accuracy: 61.9
Epoch: 26
Train accuracy: 66.54285714285714
Test accuracy: 62.36666666666667
Epoch: 27
Train accuracy: 67.55714285714286
Test accuracy: 62.53333333333333
Epoch: 28
Train accuracy: 68.42857142857143
Test accuracy: 63.0
Epoch: 29
Train accuracy: 69.22857142857143
Test accuracy: 63.23333333333333
Epoch: 30
Train accuracy: 70.05714285714286
Test accuracy: 62.8
Epoch: 31
Train accuracy: 70.74285714285715
Test accuracy: 64.36666666666667
Epoch: 32
Train accuracy: 71.37142857142858
Test accuracy: 63.866666666666674
Epoch: 33
Train accuracy: 71.57142857142857
Test accuracy: 64.13333333333333
Epoch: 34
Train accuracy: 72.0
Test accuracy: 65.0
Epoch: 35
Train accuracy: 72.65714285714286
Test accuracy: 64.60000000000001
Epoch: 36
Train accuracy: 73.85714285714286
Test accuracy: 64.93333333333334
Epoch: 37
Train accuracy: 73.71428571428571
Test accuracy: 65.33333333333333
Epoch: 38
Train accuracy: 74.18571428571428
Test accuracy: 65.4
Epoch: 39
Train accuracy: 74.47142857142856
Test accuracy: 65.2
Epoch: 40
Train accuracy: 74.97142857142856
Test accuracy: 65.16666666666666
Epoch: 41
Train accuracy: 75.58571428571429
Test accuracy: 65.8
Epoch: 42
Train accuracy: 76.28571428571429
Test accuracy: 66.16666666666666
Epoch: 43
Train accuracy: 77.24285714285715
Test accuracy: 66.60000000000001
Epoch: 44
Train accuracy: 76.71428571428571
Test accuracy: 66.10000000000001
Epoch: 45
Train accuracy: 77.8
Test accuracy: 65.86666666666666
Epoch: 46
Train accuracy: 77.82857142857142
Test accuracy: 66.8
Epoch: 47
Train accuracy: 77.52857142857142
Test accuracy: 67.03333333333333
Epoch: 48
Train accuracy: 78.44285714285715
Test accuracy: 66.93333333333334
Epoch: 49
Train accuracy: 79.35714285714286
Test accuracy: 66.63333333333334
Epoch: 50
Train accuracy: 79.02857142857142
Test accuracy: 65.96666666666667
Epoch: 51
Train accuracy: 79.5
Test accuracy: 66.56666666666666
Epoch: 52
Train accuracy: 79.25714285714285
Test accuracy: 65.93333333333334
Epoch: 53
Train accuracy: 80.11428571428571
Test accuracy: 66.46666666666667
Epoch: 54
Train accuracy: 79.85714285714286
Test accuracy: 67.43333333333334
Epoch: 55
Train accuracy: 81.02857142857142
Test accuracy: 66.23333333333333
Epoch: 56
Train accuracy: 81.07142857142857
Test accuracy: 67.23333333333333
Epoch: 57
Train accuracy: 81.77142857142857
Test accuracy: 66.53333333333333
Epoch: 58
Train accuracy: 81.55714285714286
Test accuracy: 66.86666666666666
Epoch: 59
Train accuracy: 81.41428571428571
Test accuracy: 66.46666666666667
Epoch: 60
Train accuracy: 82.32857142857142
Test accuracy: 67.53333333333333
Epoch: 61
Train accuracy: 82.55714285714286
Test accuracy: 67.66666666666666
Epoch: 62
Train accuracy: 83.47142857142858
Test accuracy: 67.13333333333334
Epoch: 63
Train accuracy: 82.98571428571428
Test accuracy: 67.10000000000001
Epoch: 64
Train accuracy: 83.51428571428572
Test accuracy: 67.2
Epoch: 65
Train accuracy: 83.6
Test accuracy: 67.13333333333334
Epoch: 66
Train accuracy: 84.44285714285714
Test accuracy: 67.03333333333333
Epoch: 67
Train accuracy: 84.61428571428571
Test accuracy: 68.0
Epoch: 68
Train accuracy: 84.57142857142857
Test accuracy: 67.36666666666666
Epoch: 69
Train accuracy: 85.57142857142857
Test accuracy: 67.23333333333333
Epoch: 70
Train accuracy: 85.9
Test accuracy: 66.8
Epoch: 71
Train accuracy: 85.55714285714285
Test accuracy: 66.43333333333334
Epoch: 72
Train accuracy: 85.77142857142857
Test accuracy: 67.26666666666667
Epoch: 73
Train accuracy: 86.95714285714286
Test accuracy: 67.30000000000001
Epoch: 74
Train accuracy: 87.05714285714285
Test accuracy: 67.60000000000001
Epoch: 75
Train accuracy: 87.55714285714285
Test accuracy: 66.93333333333334
Epoch: 76
Train accuracy: 87.4857142857143
Test accuracy: 67.36666666666666
Epoch: 77
Train accuracy: 87.82857142857144
Test accuracy: 67.66666666666666
Epoch: 78
Train accuracy: 88.34285714285714
Test accuracy: 67.26666666666667
Epoch: 79
Train accuracy: 88.71428571428571
Test accuracy: 67.83333333333333
Epoch: 80
Train accuracy: 88.44285714285715
Test accuracy: 67.10000000000001
Epoch: 81
Train accuracy: 89.0142857142857
Test accuracy: 67.56666666666666
Epoch: 82
Train accuracy: 89.6857142857143
Test accuracy: 67.0
Epoch: 83
Train accuracy: 90.07142857142857
Test accuracy: 67.36666666666666
Epoch: 84
Train accuracy: 89.7
Test accuracy: 68.83333333333333
Epoch: 85
Train accuracy: 90.41428571428571
Test accuracy: 67.30000000000001
Epoch: 86
Train accuracy: 90.81428571428572
Test accuracy: 67.76666666666667
Epoch: 87
Train accuracy: 91.72857142857143
Test accuracy: 67.80000000000001
Epoch: 88
Train accuracy: 91.37142857142857
Test accuracy: 67.86666666666666
Epoch: 89
Train accuracy: 92.10000000000001
Test accuracy: 68.10000000000001
Epoch: 90
Train accuracy: 92.0
Test accuracy: 67.93333333333334
Epoch: 91
Train accuracy: 91.97142857142858
Test accuracy: 68.30000000000001
Epoch: 92
Train accuracy: 92.2
Test accuracy: 67.80000000000001
Epoch: 93
Train accuracy: 92.54285714285714
Test accuracy: 67.43333333333334
Epoch: 94
Train accuracy: 93.07142857142857
Test accuracy: 68.4
Epoch: 95
Train accuracy: 93.18571428571428
Test accuracy: 67.63333333333334
Epoch: 96
Train accuracy: 93.28571428571428
Test accuracy: 67.60000000000001
Epoch: 97
Train accuracy: 93.75714285714287
Test accuracy: 67.43333333333334
Epoch: 98
Train accuracy: 93.38571428571429
Test accuracy: 68.33333333333333
Epoch: 99
Train accuracy: 93.78571428571428
Test accuracy: 68.13333333333334
Epoch: 100
Train accuracy: 94.54285714285714
Test accuracy: 67.93333333333334
Epoch: 101
Train accuracy: 94.3
Test accuracy: 67.9
Epoch: 102
Train accuracy: 94.41428571428571
Test accuracy: 67.80000000000001
Epoch: 103
Train accuracy: 94.84285714285714
Test accuracy: 68.43333333333334
Epoch: 104
Train accuracy: 94.6
Test accuracy: 68.06666666666666
Epoch: 105
Train accuracy: 94.78571428571428
Test accuracy: 68.5
Epoch: 106
Train accuracy: 95.11428571428571
Test accuracy: 68.13333333333334
Epoch: 107
Train accuracy: 95.5
Test accuracy: 68.86666666666666
Epoch: 108
Train accuracy: 95.71428571428572
Test accuracy: 67.80000000000001
Epoch: 109
Train accuracy: 95.87142857142858
Test accuracy: 68.33333333333333
Epoch: 110
Train accuracy: 96.14285714285714
Test accuracy: 68.03333333333333
Epoch: 111
Train accuracy: 95.84285714285714
Test accuracy: 68.53333333333333
Epoch: 112
Train accuracy: 96.1
Test accuracy: 68.83333333333333
Epoch: 113
Train accuracy: 95.72857142857143
Test accuracy: 68.56666666666666
Epoch: 114
Train accuracy: 96.42857142857143
Test accuracy: 68.5
Epoch: 115
Train accuracy: 96.64285714285714
Test accuracy: 69.66666666666667
Epoch: 116
Train accuracy: 96.52857142857142
Test accuracy: 68.4
Epoch: 117
Train accuracy: 97.04285714285714
Test accuracy: 68.7
Epoch: 118
Train accuracy: 96.72857142857143
Test accuracy: 68.53333333333333
Epoch: 119
Train accuracy: 96.78571428571429
Test accuracy: 68.46666666666667
Epoch: 120
Train accuracy: 97.35714285714285
Test accuracy: 68.63333333333334
Epoch: 121
Train accuracy: 97.14285714285714
Test accuracy: 68.26666666666667
Epoch: 122
Train accuracy: 97.5142857142857
Test accuracy: 68.2
Epoch: 123
Train accuracy: 97.2
Test accuracy: 68.26666666666667
Epoch: 124
Train accuracy: 97.47142857142858
Test accuracy: 68.63333333333334
Epoch: 125
Train accuracy: 97.72857142857143
Test accuracy: 67.80000000000001
Epoch: 126
Train accuracy: 97.8142857142857
Test accuracy: 68.89999999999999
Epoch: 127
Train accuracy: 97.75714285714285
Test accuracy: 68.7
Epoch: 128
Train accuracy: 98.17142857142858
Test accuracy: 68.53333333333333
Epoch: 129
Train accuracy: 98.25714285714285
Test accuracy: 68.26666666666667
Epoch: 130
Train accuracy: 98.22857142857143
Test accuracy: 69.26666666666667
Epoch: 131
Train accuracy: 98.2
Test accuracy: 68.16666666666666
Epoch: 132
Train accuracy: 98.35714285714286
Test accuracy: 69.26666666666667
Epoch: 133
Train accuracy: 98.18571428571428
Test accuracy: 68.2
Epoch: 134
Train accuracy: 98.5
Test accuracy: 68.89999999999999
Epoch: 135
Train accuracy: 98.55714285714285
Test accuracy: 68.30000000000001
Epoch: 136
Train accuracy: 98.68571428571428
Test accuracy: 68.13333333333334
Epoch: 137
Train accuracy: 98.65714285714286
Test accuracy: 68.8
Epoch: 138
Train accuracy: 98.4
Test accuracy: 67.63333333333334
Epoch: 139
Train accuracy: 98.6
Test accuracy: 68.93333333333334
Epoch: 140
Train accuracy: 98.58571428571429
Test accuracy: 68.76666666666667
Epoch: 141
Train accuracy: 98.45714285714286
Test accuracy: 67.73333333333333
Epoch: 142
Train accuracy: 98.58571428571429
Test accuracy: 68.56666666666666
Epoch: 143
Train accuracy: 98.81428571428572
Test accuracy: 67.86666666666666
Epoch: 144
Train accuracy: 98.81428571428572
Test accuracy: 68.5
Epoch: 145
Train accuracy: 98.58571428571429
Test accuracy: 69.53333333333333
Epoch: 146
Train accuracy: 98.9857142857143
Test accuracy: 68.56666666666666
Epoch: 147
Train accuracy: 98.91428571428571
Test accuracy: 68.89999999999999
Epoch: 148
Train accuracy: 99.12857142857143
Test accuracy: 68.63333333333334
Epoch: 149
Train accuracy: 99.05714285714285
Test accuracy: 68.53333333333333
Epoch: 150
Train accuracy: 99.11428571428571
Test accuracy: 68.43333333333334
Epoch: 151
Train accuracy: 99.0
Test accuracy: 69.03333333333333
Epoch: 152
Train accuracy: 99.18571428571428
Test accuracy: 68.63333333333334
Epoch: 153
Train accuracy: 99.12857142857143
Test accuracy: 68.23333333333333
Epoch: 154
Train accuracy: 99.07142857142858
Test accuracy: 68.56666666666666
Epoch: 155
Train accuracy: 99.3
Test accuracy: 69.03333333333333
Epoch: 156
Train accuracy: 99.3
Test accuracy: 68.89999999999999
Epoch: 157
Train accuracy: 99.4
Test accuracy: 68.89999999999999
Epoch: 158
Train accuracy: 99.11428571428571
Test accuracy: 69.13333333333334
Epoch: 159
Train accuracy: 99.18571428571428
Test accuracy: 69.06666666666666
Epoch: 160
Train accuracy: 99.3
Test accuracy: 68.43333333333334
Epoch: 161
Train accuracy: 99.45714285714286
Test accuracy: 68.36666666666666
Epoch: 162
Train accuracy: 99.41428571428571
Test accuracy: 69.96666666666667
Epoch: 163
Train accuracy: 99.6
Test accuracy: 69.03333333333333
Epoch: 164
Train accuracy: 99.35714285714286
Test accuracy: 69.26666666666667
Epoch: 165
Train accuracy: 99.54285714285714
Test accuracy: 69.3
Epoch: 166
Train accuracy: 99.47142857142856
Test accuracy: 68.46666666666667
Epoch: 167
Train accuracy: 99.22857142857143
Test accuracy: 69.3
Epoch: 168
Train accuracy: 99.5142857142857
Test accuracy: 69.63333333333334
Epoch: 169
Train accuracy: 99.6
Test accuracy: 69.26666666666667
Epoch: 170
Train accuracy: 99.47142857142856
Test accuracy: 69.5
Epoch: 171
Train accuracy: 99.4857142857143
Test accuracy: 69.83333333333334
Epoch: 172
Train accuracy: 99.65714285714286
Test accuracy: 69.03333333333333
Epoch: 173
Train accuracy: 99.64285714285714
Test accuracy: 68.63333333333334
Epoch: 174
Train accuracy: 99.57142857142857
Test accuracy: 68.89999999999999
Epoch: 175
Train accuracy: 99.68571428571428
Test accuracy: 69.93333333333334
Epoch: 176
Train accuracy: 99.6
Test accuracy: 68.89999999999999
Epoch: 177
Train accuracy: 99.62857142857143
Test accuracy: 68.83333333333333
Epoch: 178
Train accuracy: 99.55714285714285
Test accuracy: 69.8
Epoch: 179
Train accuracy: 99.74285714285715
Test accuracy: 69.76666666666667
Epoch: 180
Train accuracy: 99.72857142857143
Test accuracy: 69.33333333333334
Epoch: 181
Train accuracy: 99.7
Test accuracy: 68.86666666666666
Epoch: 182
Train accuracy: 99.71428571428571
Test accuracy: 69.86666666666666
Epoch: 183
Train accuracy: 99.75714285714285
Test accuracy: 69.96666666666667
Epoch: 184
Train accuracy: 99.74285714285715
Test accuracy: 69.66666666666667
Epoch: 185
Train accuracy: 99.65714285714286
Test accuracy: 69.6
Epoch: 186
Train accuracy: 99.7
Test accuracy: 70.23333333333333
Epoch: 187
Train accuracy: 99.67142857142856
Test accuracy: 69.39999999999999
Epoch: 188
Train accuracy: 99.81428571428572
Test accuracy: 68.83333333333333
Epoch: 189
Train accuracy: 99.77142857142857
Test accuracy: 68.13333333333334
Epoch: 190
Train accuracy: 99.82857142857144
Test accuracy: 68.56666666666666
Epoch: 191
Train accuracy: 99.61428571428571
Test accuracy: 68.2
Epoch: 192
Train accuracy: 99.75714285714285
Test accuracy: 69.16666666666667
Epoch: 193
Train accuracy: 99.74285714285715
Test accuracy: 68.66666666666667
Epoch: 194
Train accuracy: 99.74285714285715
Test accuracy: 69.0
Epoch: 195
Train accuracy: 99.75714285714285
Test accuracy: 68.03333333333333
Epoch: 196
Train accuracy: 99.82857142857144
Test accuracy: 68.73333333333333
Epoch: 197
Train accuracy: 99.82857142857144
Test accuracy: 68.06666666666666
Epoch: 198
Train accuracy: 99.72857142857143
Test accuracy: 69.33333333333334
Epoch: 199
Train accuracy: 99.65714285714286
Test accuracy: 69.69999999999999

In [0]:
model

ConvNet(
  (drop): Dropout2d(p=0.05)
  (dropfc): Dropout(p=0.2)
  (conv1): Conv2d(1, 128, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(256, 256, kernel_size=(5, 5), stride=(1, 1))
  (conv4): Conv2d(256, 128, kernel_size=(5, 5), stride=(1, 1))
  (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm2d(92, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn6): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3200, out_features=1024, bias=Tru

In [0]:
files.download(r'final_submission.csv')